# Hypothesis: Property-based testing

In this notebook we use property-based testing to find problems in our code. [Hypothesis](https://hypothesis.readthedocs.io/en/latest/) is a library similar to Haskell’s [Quickcheck](https://hackage.haskell.org/package/QuickCheck). We’ll get to know it in more detail later, along with other test libraries: Hypothesis. [Hypothesis](https://jupyter-tutorial.readthedocs.io/en/latest/notebook/testing/hypothesis.html) can also provide mock objects and tests for numpy data types.

## 1. Imports

In [1]:
import re

from hypothesis import assume, given
from hypothesis.strategies import emails, integers, tuples

## 2. Find range

In [2]:
def calculate_range(tuple_obj):
    return max(tuple_obj) - min(tuple_obj)

## 3. Test with `strategies` and `given`

With [hypothesis.strategies](https://hypothesis.readthedocs.io/en/latest/data.html) you can create different test data. For this, Hypothesis provides strategies for most types and arguments restrict the possibilities to suit your needs. In the example below, we use the [integers](https://hypothesis.readthedocs.io/en/latest/data.html#hypothesis.strategies.integers) strategy, which is applied to the function with the [Python-Decorator](https://docs.python.org/3/glossary.html#term-decorator) `@given`. More specifically, it takes our test function and converts it into a parameterised one to run over wide ranges of matching data:

In [3]:
@given(tuples(integers(), integers(), integers()))
def test_calculate_range(tup):
    result = calculate_range(tup)
    assert isinstance(result, int)
    assert result > 0

In [4]:
test_calculate_range()

AssertionError: 

Now we correct the test with `>=` and check it again:

In [5]:
@given(tuples(integers(), integers()))
def test_calculate_range(tup):
    result = calculate_range(tup)
    assert isinstance(result, int)
    assert result >= 0

In [6]:
test_calculate_range()

## 3. Check against regular expressions

[Regular expressions](https://en.wikipedia.org/wiki/Regular_expression) can be used to check strings for certain syntactical rules. In Python, you can use [re.match](https://docs.python.org/3/library/re.html#re.match) to check regular expressions.

<div class="alert alert-block alert-info">

**Note:**

On the website [regex101](https://regex101.com/) you can first try out your regular expressions.
</div>

As an example, let’s try to find out the `username` and the `domain` from email addresses:

In [7]:
def parse_email(email):
    result = re.match(
        r"(?P<username>\w+).(?P<domain>[\w\.]+)",
        email,
    ).groups()
    return result

Now we write a test `test_parse_email` to check our method. As input values we use the [emails](https://hypothesis.readthedocs.io/en/latest/data.html#hypothesis.strategies.emails) strategy of Hypothesis. As result we expect for example:
```
('0', 'A.com')
('F', 'j.EeHNqsx')
…
```
In the test, we assume on the one hand that two entries are always returned and that a dot (`.`) occurs in the second entry. 

In [8]:
@given(emails())
def test_parse_email(email):
    result = parse_email(email)
    # print(result)
    assert len(result) == 2
    assert "." in result[1]                                                                                                                                      

In [9]:
test_parse_email()

  + Exception Group Traceback (most recent call last):
  |   File "/Users/veit/sandbox/py313/.venv/lib/python3.13/site-packages/IPython/core/interactiveshell.py", line 3577, in run_code
  |     exec(code_obj, self.user_global_ns, self.user_ns)
  |     ~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  |   File "/var/folders/hk/s8m0bblj0g10hw885gld52mc0000gn/T/ipykernel_6234/3120039226.py", line 1, in <module>
  |     test_parse_email()
  |     ~~~~~~~~~~~~~~~~^^
  |   File "/var/folders/hk/s8m0bblj0g10hw885gld52mc0000gn/T/ipykernel_6234/102862131.py", line 2, in test_parse_email
  |     def test_parse_email(email):
  |                    ^^^
  |   File "/Users/veit/sandbox/py313/.venv/lib/python3.13/site-packages/hypothesis/core.py", line 1705, in wrapped_test
  |     raise the_error_hypothesis_found
  | ExceptionGroup: Hypothesis found 2 distinct failures. (2 sub-exceptions)
  +-+---------------- 1 ----------------
    | Traceback (most recent call last):
    |   File "/var/folders/h

With Hypothesis, two examples were found that make it clear that our regular expression in the `parse_email` method is not yet sufficient: `0/0@A.ac` and `/@A.ac`. After we have adapted our regular expression accordingly, we can call the test again:

In [10]:
def parse_email(email):
    result = re.match(
        r"(?P<username>[\.\w\-\!~#$%&\|{}\+\/\^\`\=\*']+).(?P<domain>[\w\.\-]+)",
        email,
    ).groups()
    return result

In [11]:
test_parse_email()